<a href="https://colab.research.google.com/github/megha1906/TransferLearning-using-VGG16/blob/master/Transfer_Learning_using_vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.applications import vgg16

# MobileNet was designed to work on 224 x 224 pixel input images sizes
img_rows, img_cols = 224, 224 

# Re-loads the MobileNet model without the top or FC layers
vgg = vgg16.VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in vgg.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(vgg.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)
    
 
def lw(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model 
    
    
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from keras.models import Model

# Set our class number to 3 (Young, Middle, Old)
num_classes = 2

FC_Head = lw(vgg, num_classes)

model = Model(inputs = vgg.input, outputs = FC_Head)

print(model.summary())

2023-11-27 10:24:39.426371: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-27 10:24:39.508698: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-27 10:24:39.508945: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-27 10:24:39.512882: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-27 10:24:39.528612: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-27 10:24:39.530199: I tensorflow/core/platform/cpu_feature_guard.cc:1

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                               

In [2]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'photos/photos/train'
validation_data_dir = 'photos/photos/validate'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# set our batch size (typically on most mid tier systems we'll use 16-32)
batch_size = 32
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')
        
        
        

Found 268 images belonging to 2 classes.
Found 250 images belonging to 2 classes.


In [3]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("face recognisation.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 108
nb_validation_samples = 52

# We only train 5 EPOCHS 
epochs = 5
batch_size = 16

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)
    

/tmp/ipykernel_16021/84430091.py:33: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/5


2023-11-27 10:24:44.329025: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2023-11-27 10:24:44.502517: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2023-11-27 10:24:45.244728: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2023-11-27 10:24:45.458716: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.


1/6 [====>.........................] - ETA: 26s - loss: 0.7123 - accuracy: 0.2812

2023-11-27 10:24:48.564417: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.


6/6 [==============================] - ETA: 0s - loss: 1.4953 - accuracy: 0.5156
Epoch 1: val_loss improved from inf to 0.56968, saving model to face recognisation.h5


/home/raihan/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


6/6 [==============================] - 39s 7s/step - loss: 1.4953 - accuracy: 0.5156 - val_loss: 0.5697 - val_accuracy: 0.6354
Epoch 2/5
6/6 [==============================] - ETA: 0s - loss: 0.5765 - accuracy: 0.7500
Epoch 2: val_loss did not improve from 0.56968
6/6 [==============================] - 35s 7s/step - loss: 0.5765 - accuracy: 0.7500 - val_loss: 0.7994 - val_accuracy: 0.3854
Epoch 3/5
6/6 [==============================] - ETA: 0s - loss: 0.3568 - accuracy: 0.8140
Epoch 3: val_loss improved from 0.56968 to 0.50427, saving model to face recognisation.h5
6/6 [==============================] - 67s 13s/step - loss: 0.3568 - accuracy: 0.8140 - val_loss: 0.5043 - val_accuracy: 0.7812
Epoch 4/5
6/6 [==============================] - ETA: 0s - loss: 0.2930 - accuracy: 0.8837
Epoch 4: val_loss improved from 0.50427 to 0.48686, saving model to face recognisation.h5
6/6 [==============================] - 67s 12s/step - loss: 0.2930 - accuracy: 0.8837 - val_loss: 0.4869 - val_accurac

In [3]:
from keras.models import load_model
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

classifier = load_model('face recognisation.h5')

monkey_breeds_dict = {"[0]": " raihan ", "[1]": "aamir "}
monkey_breeds_dict_n = {"n0": " raihan", "n1": "aamir "}

def draw_test(name, pred, im):
    monkey = monkey_breeds_dict[str(pred)]
    BLACK = [0, 0, 0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100, cv2.BORDER_CONSTANT, value=BLACK)
    cv2.putText(expanded_image, monkey, (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow(name, expanded_image)
    cv2.waitKey(0)



def print_prediction(pred):
    monkey = monkey_breeds_dict[str(pred)]
    print("Predicted class: {}".format(monkey))

def getRandomImage(path):
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0, len(folders))
    path_class = folders[random_directory]
    print("Class - " + monkey_breeds_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0, len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path + "/" + image_name)

for i in range(0, 10):
    input_im = getRandomImage("photos/photos/validate/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)

    input_im = cv2.resize(input_im, (224, 224), interpolation=cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1, 224, 224, 3)
    
    # Draw prediction
    # draw_test("Prediction", res, input_original)

    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose=0), axis=1)

    # Print the predicted class
    print_prediction(res)


Class - aamir 
Predicted class: aamir 
Class -  raihan
Predicted class:  raihan 
Class - aamir 
Predicted class: aamir 
Class -  raihan
Predicted class: aamir 
Class - aamir 
Predicted class: aamir 
Class -  raihan
Predicted class:  raihan 
Class -  raihan
Predicted class:  raihan 
Class -  raihan
Predicted class:  raihan 
Class - aamir 
Predicted class: aamir 
Class - aamir 
Predicted class: aamir 
